### Adding elevation data to trajectories from NED13 tiles

Creating the cypy2env:

`conda create -n cypyenv -c conda-forge python=3.7 psycopg2 rasterio pandas ipykernel scipy geopy seaborn`

Then, use pip to install matplotlib, fitparse, and gitpython.

In [ ]:
import os
import re
import sys
import glob
import time
import json
import pickle
import datetime
import psycopg2
import subprocess
import rasterio

import geopy
import numpy as np
import pandas as pd
import seaborn as sns

from psycopg2 import sql
from scipy import interpolate

import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
import rasterio.warp
import rasterio.windows
import rasterio.enums
import geopy.distance

In [ ]:
from cycler import cycler
red, blue, green, purple, orange, yellow, brown, pink, gray  = sns.color_palette('Set1')
mpl.rcParams['axes.prop_cycle'] = cycler(color=[blue, orange, green, red, brown, gray])

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
sys.path.insert(0, '../../dbutils/')
import dbutils

sys.path.insert(0, '../')
import cypy2

colors = sns.color_palette()

root = '/home/keith/Downloads/export_7989839-1'
wahoo_example = '2326365683.fit.gz'
garmin_example = '2122584483.fit.gz'
garmin_indoor_example = '2324139976.fit.gz'

### Connect to OSM roads database

This database is created/populated by `strava/osm_to_pgsql.sh`.

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'osm_roads'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
# king ridge road
osm_id = '7712776'
d = dbutils.execute_query(
    conn, 
    'select ST_AsGeoJSON(geom) from roads where osm_id = %s', osm_id)

In [ ]:
d = json.loads(d[0][0])
kr_osm_coords = np.array(d['coordinates'][0])

In [ ]:
plt.scatter(coords[:, 0], coords[:, 1])

### Creating a postGIS table of Strava routes from GPX files

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'routes'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
from cypy2.strava import strava_routes_to_pgsql

In [ ]:
filenames = glob.glob('/home/keith/Downloads/export_7989839-1/routes/*.gpx')
strava_routes_to_pgsql.insert_routes(conn, 'routes', filenames)

In [ ]:
# metadata from tracks table
d = pd.read_sql(
    'select track_id, type, name from tracks where name like \'%King%\'', conn)
d.head()

In [ ]:
# coords from the track_points table
d = pd.read_sql('''
    select track_id, ele, ST_AsGeoJSON(geom) as geom 
    from track_points where track_id = 324 order by point_order''', conn)

d['lon'] = [json.loads(geom)['coordinates'][0] for geom in d.geom]
d['lat'] = [json.loads(geom)['coordinates'][1] for geom in d.geom]

d.drop('geom', axis=1, inplace=True)
d.head()

In [ ]:
# coords from geom column
plt.scatter(d.lon, d.lat)

### Interpolating routes by distance

In [ ]:
lat, lon = d.lat.values, d.lon.values

In [ ]:
# distance between adjacent coordinates
dists = [0]
for ind in np.arange(1, lat.shape[0]):
    dist = geopy.distance.distance(
        (lat[ind-1], lon[ind-1]), 
        (lat[ind], lon[ind]))
    dists.append(dist.meters)

d['dist'] = np.cumsum(dists)

In [ ]:
plt.scatter(lat, lon, s=np.array(dists)**2/100)

In [ ]:
# the distances are all between 8m and 100m 
# (while there appears to be a hard cut-off at 100m,
# the 8m is not a hard cut-off and is likely due to the tolerance in the rdp algorithm)
plt.hist(dists, bins=300)

In [ ]:
# interpolate lat/lon coords by distance (which is approximately by arc length)
f = interpolate.interp1d(d.dist.values, d[['lat', 'lon']].values, axis=0, kind='cubic')
latlon = f(np.arange(0, d.dist.max(), 10))

In [ ]:
plt.scatter(lat, lon, s=30)
plt.plot(latlon[:, 0], latlon[:, 1])

In [ ]:
dists = []
for ind in np.arange(1, vals_int.shape[0]):
    dists.append(geopy.distance.distance(latlon[ind-1, :], latlon[ind, :]).meters)
dists = np.array(dists)

In [ ]:
plt.hist(dists, bins=np.arange(9, 11, .01))

In [ ]:
# these large jumps in the interpolated coordinates 
# are associated with tiny but sharp loops in the original track
# (due, in at least one case, to a waypoint being placed 
# on the wrong side of the road in the strava route builder)
np.argwhere(np.array(dists) > 15)

### Looking up elevations from NED13 tiles

In [ ]:
src = rasterio.open(source_path)

In [ ]:
px, py = src.index(*tam)

(
# lon/lat of the pixel's top left corner
src.transform * (py, px), 

# lon/lat of the pixel's bottom right corner
src.transform * (py + 1, px + 1)
)

In [ ]:
res = 9.2592592593e-05

# lon/lat of the point at which to interpolate the elevation
tam = np.array([-122.59685, 37.92301])

def interp_elevation(src, point, kind='cubic', sz=2):

    # row/column of the pixel containing the point
    row, col = src.index(*point)

    # lon/lat of the top left corner of this pixel
    px_lon, px_lat = src.transform * (col, row)

    # coordinates of the centers of the window pixels
    x = np.arange(-sz, sz + 1) * res + px_lon + res/2
    y = -(px_lat - np.arange(-sz, sz + 1) * res) + res/2

    window = rasterio.windows.Window(col - sz, row - sz, 2*sz + 1, 2*sz + 1)
    values = src.read(window=window, masked=False)
    z = values[0]
    
    # note: do not flatten z before passing to interp2d!
    # for some reason, the interpolated values change (and are incorrect)
    return interpolate.interp2d(x, y, z, kind='cubic', bounds_error=True)(point[0], -point[1])

In [ ]:
interp_elevation(src, tam)

In [ ]:
plt.imshow(data[0, :, :])

In [ ]:
latlon.min(axis=0), latlon.max(axis=0)

In [ ]:
source_path = '/home/keith/raster-data/n38w123/grdn38w123_13/w001001.adf'
source_path = '/media/keith/USGS_Backup/USGS/NED13/n38w119/grdn38w119_13/w001001.adf'
d['ned13_raw'] = None
d['ned13_int'] = None

with rasterio.open(source_path) as src:
    for ind, row in d.iterrows():
        point = (row.lon, row.lat)
        d.at[ind, 'ned13_int'] = interp_elevation(src, point, sz=3)
        d.at[ind, 'ned13_raw'] = [v for v in src.sample([point], indexes=src.indexes)][0]

### Compare our elevations to the original elevations (from Strava)

In [ ]:
plt.plot(d.ned13_int.values.astype(float))
plt.plot(np.convolve(d.ned13_int.values.astype(float), np.ones(5)/5, 'same'))
plt.plot(d.ele.values.astype(float))

In [ ]:
# the resolution in meters of the NED13 tiles near SF
x = rasterio.warp.transform('EPSG:4269', 'EPSG:3857', [-122.5 + res, -122.5], [37.7, 37.7 + res])
np.diff(x[0]), np.diff(x[1])

In [ ]:
plt.plot(d.dist.values, d.ele.values.astype(float), marker='o')